## PH model
@author: Or Duek

In this notebook there are two different ways of calculating the Hybrid model
On each function you'll see reference to the relevant paper

In [3]:
%config Completer.use_jedi = False

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import theano
import theano.tensor as tt
import scipy
import pingouin as pg
import os

import pymc3 as pm
import arviz as az
import statsmodels.api as sm
import statsmodels.formula.api as smf

from glob import glob

#### Load data (Nachshon)

In [4]:
#Load Age and MoCA scores from the session log file.
age = pd.read_excel('/media/Data/Lab_Projects/Aging/aging_session_log.xlsx', sheet_name='sessions').iloc[:,[0,7,9]]
moca = pd.read_excel('/media/Data/Lab_Projects/Aging/aging_session_log.xlsx', sheet_name='assessments').iloc[:,[0,22]]

# Change name of the first column to 'sub'
age.columns.values[0] = moca.columns.values[0] = "sub"

# remove unnecessary rows
age = age[age.gender != 'gender']
age = age.dropna().reset_index()
age = age.drop(['index'], axis=1)

print("Number of participants with age: ",age.shape[0],'; females: ', age[age['gender']=='F'].shape[0])
# merge files
age = age.merge(moca, left_on='sub', right_on='sub')
age['sub'] = age['sub'].map(lambda x: int(x.lstrip('AG_'))) # remove the AG_ prfix from the subject number
age['sub'] = age['sub'].astype(int)
age['age'] = age.age.astype('int')

print('Number of participants with MoCA: ', age.shape[0], '; MoCA > 25:', age[age.moca_score>25].shape[0],'; females: ', age[age['gender']=='F'].shape[0])
print('mean:', np.mean(age.age), 'std:', np.std(age.age), 'min-max:',np.min(age.age),np.max(age.age))
age_c = age[age['moca_score']>25]
print('mean:', np.mean(age_c.age), 'std:', np.std(age_c.age), 'min-max:',np.min(age_c.age),np.max(age_c.age))

Number of participants with age:  91 ; females:  43
Number of participants with MoCA:  81 ; MoCA > 25: 63 ; females:  38
mean: 54.45679012345679 std: 22.041514916632877 min-max: 18 89
mean: 51.888888888888886 std: 21.998637123089434 min-max: 18 88


In [5]:
glober = '/media/Data/Lab_Projects/Aging/behavioral/Reversal/AG_*_RV/ETLearning_*.csv'

db = pd.DataFrame()

for sub in glob(glober):
    
    try:
        df = pd.read_csv(sub)
        df['sub'] = sub.split('_')[2]
        if df.shape[0] == 70:
            db = pd.concat([db, df], axis = 0)
            #db = db.append(df)#[df.trialNum<36])
    except:
        print(sub)
        print('error')

#db['rating'] = db['rating'].replace(0, np.nan)
db = db.sort_values(by=['sub','trialNum'])
print('number of subject: ', len(db['sub'].unique()))

number of subject:  88


In [6]:
db['sub'] = db['sub'].astype('int')
db = db.merge(age_c, left_on='sub', right_on='sub')
#db = db[db.moca_score >25]
db = db.sort_values('sub')
print('Valid subjects: ', len(np.unique(db['sub'])))

Valid subjects:  63


In [7]:
x = db[db['RT']==-1].groupby('sub').count()['rating']
x = x[x>5].reset_index()
print(x)
db = db[~db['sub'].isin(x['sub'].values)]

   sub  rating
0  102      13


In [8]:
subs = db.drop_duplicates('sub')
subs = subs.drop(['trialNum', 'rectOri', 'rectValue', 'rating', 'RT'],axis = 1).reset_index()
#subs.head()

## get descriptive data

In [17]:
n_subj   = len(db['sub'].unique())
n_trials = max(db.trialNum)

trials, subj = np.meshgrid(range(n_trials), range(n_subj))
trials = tt.as_tensor_variable(trials.T)
subj   = tt.as_tensor_variable(subj.T)

In [18]:
stim   = np.reshape([db['rectOri']],   (n_subj, n_trials)).T
reward = np.reshape([db['rectValue']], (n_subj, n_trials)).T
rating = np.reshape([db['rating']],    (n_subj, n_trials)).T

stim   = np.array(stim/45,  dtype='int')
reward = np.array(reward/6*9, dtype='int')

In [22]:
n_subj

62

In [19]:
stim = tt.as_tensor_variable(stim)
reward = tt.as_tensor_variable(reward)

In [60]:
# Rouhani et al. version https://elifesciences.org/articles/61077
# generate functions to run
def update_Q_r(stim, shock,
             Qs,vec,
             eta,kappa,n_subj):
    """
    This function updates the Q table according to Hybrid PH model
    For information, please see this paper: https://www.sciencedirect.com/science/article/pii/S0896627316305840?via%3Dihub
  
    """
      
    delta = shock - Qs[tt.arange(n_subj), stim]
    #alpha = tt.set_subtensor(alpha[tt.arange(n_subj), stim], eta * abs(delta) + (1-eta)*alpha[tt.arange(n_subj), stim])
    # based on Rohani et al. 
    #alpha = tt.set_subtensor(alpha[tt.arange(n_subj), stim], eta + kappa * abs(delta))
    alpha = eta + (kappa * abs(delta))
    alpha = 1 / (1 + pm.math.exp(-alpha)) # sigmoid function
    Qs = tt.set_subtensor(Qs[tt.arange(n_subj),stim], Qs[tt.arange(n_subj),stim] + alpha * delta)
    
    # in order to get a vector of expected outcome (dependent on the stimulus presentes [CS+, CS-] 
    # we us if statement (switch in theano)
    vec = tt.set_subtensor(vec[tt.arange(n_subj),0], (tt.switch(tt.eq(stim,1), 
                                                                Qs[tt.arange(n_subj),1], Qs[tt.arange(n_subj),0])))
    
    # we use the same idea to get the associability per trial
    # assoc = tt.set_subtensor(assoc[tt.arange(n_subj),0], (tt.switch(tt.eq(stim,1), 
    #                                                             alpha[tt.arange(n_subj),1], alpha[tt.arange(n_subj),0])))
    
    return Qs, vec, alpha

In [ ]:
with pm.Model() as m1:
  
      
    eta = pm.Normal('eta', 0,1, shape=n_subj)
    kappa = pm.Normal('kappa', 0,1, shape=n_subj)
    beta = pm.Normal('beta',0, 1, shape=n_subj)
    eps = pm.HalfNormal('eps', 5)
    
    # intercept
    #intercept = pm.Normal('intercept', 0, .5, shape=n_subj)
    
    Qs = 0.5 * tt.ones((n_subj,2), dtype='float64') # set values for boths stimuli (CS+, CS-)
    vec = 0.5 * tt.ones((n_subj,1), dtype='float64') # vector to save the relevant stimulus's expactation
  #  alpha = 0 * tt.ones((n_subj,2), dtype='float64') # vector to save the relevant stimulus's expactation
   # assoc = 0 * tt.ones((n_subj,1), dtype='float64') # vector to save the relevant stimulus's expactation
    
    [Qs,vec, alpha], updates = theano.scan(
        fn=update_Q_r,
        sequences=[stim, reward],
        outputs_info=[Qs, vec, None],
        non_sequences=[eta, kappa, n_subj])
    
    vec_ = vec[trials, subj,0] * beta[subj]# + intercept[subj]
    
    scrs = pm.Normal('scrs', mu = vec_, sd = eps, observed=rating) 
    
    # add matrix of expected values (trials X subjects)
    ev = pm.Deterministic('expected_value', vec_)
    # add associability
   # alpha = pm.Deterministic('pe', alpha)
    
    tr = pm.sample(target_accept=.9, chains=4, cores=10, return_inferencedata=True)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 10 jobs)
NUTS: [eps, beta, kappa, eta]
